In [5]:
import pandas as pd


In [6]:
df = pd.read_csv("data/preprocessed_2025.csv")
df.head()

C:\Users\cucut\AppData\Local\Temp\ipykernel_7424\864629535.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/preprocessed_2025.csv")


,_id,INCIDENT_NUMBER,Initial_CAD_Event_Type,Initial_CAD_Event_Call_Type,Final_Incident_Type,Event_Alarm_Level,Call_Source,Incident_Station_Area,Incident_Ward,Ward_At_Event_Dispatch,...,TFS_Alarm_Time,TFS_Arrival_Time,Last_TFS_Unit_Clear_Time,Persons_Rescued,geometry,incident_datetime,year,month,hour,day_of_week
0,1,F23046527,Medical,Medical,34 - Human - Perceived Emergency,0,03 - From Ambulance,345,9,NaN,...,2023-04-10T18:36:49,2023-04-10T18:40:18,2023-04-10T18:59:09,0.0,"{""type"": ""Point"", ""coordinates"": [0.0, 0.0]}",2023-04-10 18:36:49,2023,4,18,0
1,2,F23048784,Medical,Medical,Medical,0,03 - From Ambulance,443,2,NaN,...,2023-04-14T20:46:32,2023-04-14T20:52:01,2023-04-14T21:06:45,0.0,"{""type"": ""Point"", ""coordinates"": [0.0, 0.0]}",2023-04-14 20:46:32,2023,4,20,4
2,3,F23042208,FIHR - Fire - Highrise Residential,Emergency Fire,01 - Fire,0,01 - 911,115,17,NaN,...,2023-04-01T17:31:49,2023-04-01T17:35:40,2023-04-01T18:33:45,0.0,"{""type"": ""Point"", ""coordinates"": [-79.3431, 43...",2023-04-01 17:31:49,2023,4,17,5
3,4,F23103667,Medical,Medical,Medical,0,03 - From Ambulance,332,10,NaN,...,2023-07-23T12:10:40,2023-07-23T12:15:15,2023-07-23T12:20:27,0.0,"{""type"": ""Point"", ""coordinates"": [0.0, 0.0]}",2023-07-23 12:10:40,2023,7,12,6
4,5,F23178577,Medical,Medical,Medical,0,03 - From Ambulance,312,11,NaN,...,2023-12-02T19:14:19,2023-12-02T19:19:22,2023-12-02T19:22:35,0.0,"{""type"": ""Point"", ""coordinates"": [0.0, 0.0]}",2023-12-02 19:14:19,2023,12,19,5


In [7]:
df.dtypes

_id                              int64
INCIDENT_NUMBER                 object
Initial_CAD_Event_Type          object
Initial_CAD_Event_Call_Type     object
Final_Incident_Type             object
Event_Alarm_Level                int64
Call_Source                     object
Incident_Station_Area           object
Incident_Ward                    int64
Ward_At_Event_Dispatch         float64
Intersection                    object
TFS_Alarm_Time                  object
TFS_Arrival_Time                object
Last_TFS_Unit_Clear_Time        object
Persons_Rescued                float64
geometry                        object
incident_datetime               object
year                             int64
month                            int64
hour                             int64
day_of_week                      int64
dtype: object

In [8]:
numerical_cols = [
    '_id',
    'Event_Alarm_Level', 
    'Incident_Ward',
    'Ward_At_Event_Dispatch',
    'Persons_Rescued',
    'year',
    'month',
    'hour',
    'day_of_week'
]

In [9]:
def remove_outliers_iqr_independent(df, columns):
    df_clean = df.copy()
    
    print(f"Original dataset size: {len(df_clean)}")
    
    for col in columns:
        original_size = len(df_clean)
        
        # Skip if column has too few non-null values
        if df_clean[col].count() < 4:
            print(f"Column {col}: Skipped - too few non-null values")
            continue
            
        # Calculate quartiles
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        
        # Skip if IQR is 0 (all values are the same)
        if IQR == 0:
            print(f"Column {col}: Skipped - IQR is 0")
            continue
        
        # Define outlier bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count outliers before removal
        outliers_mask = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
        outliers_count = outliers_mask.sum()
        
        print(f"Column: {col}")
        print(f"  Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
        print(f"  Bounds: [{lower_bound:.2f}, {upper_bound:.2f}]")
        print(f"  Outliers found: {outliers_count}")
        
        if original_size > 0:
            print(f"  Percentage: {(outliers_count/original_size)*100:.2f}%")
        
        # Remove outliers for this column
        df_clean = df_clean[~outliers_mask]
    
    return df_clean

df_cleaned_independent = remove_outliers_iqr_independent(df, numerical_cols)

Original dataset size: 361717
Column: _id
  Q1: 90430.00, Q3: 271288.00, IQR: 180858.00
  Bounds: [-180857.00, 542575.00]
  Outliers found: 0
  Percentage: 0.00%
Column Event_Alarm_Level: Skipped - IQR is 0
Column: Incident_Ward
  Q1: 7.00, Q3: 17.00, IQR: 10.00
  Bounds: [-8.00, 32.00]
  Outliers found: 0
  Percentage: 0.00%
Column Ward_At_Event_Dispatch: Skipped - too few non-null values
Column Persons_Rescued: Skipped - IQR is 0
Column: year
  Q1: 2023.00, Q3: 2024.00, IQR: 1.00
  Bounds: [2021.50, 2025.50]
  Outliers found: 0
  Percentage: 0.00%
Column: month
  Q1: 4.00, Q3: 10.00, IQR: 6.00
  Bounds: [-5.00, 19.00]
  Outliers found: 0
  Percentage: 0.00%
Column: hour
  Q1: 9.00, Q3: 18.00, IQR: 9.00
  Bounds: [-4.50, 31.50]
  Outliers found: 0
  Percentage: 0.00%
Column: day_of_week
  Q1: 1.00, Q3: 5.00, IQR: 4.00
  Bounds: [-5.00, 11.00]
  Outliers found: 0
  Percentage: 0.00%


In [10]:
import pandas as pd

# Convert to datetime
df['TFS_Alarm_Time'] = pd.to_datetime(df['TFS_Alarm_Time'], errors='coerce')
df['TFS_Arrival_Time'] = pd.to_datetime(df['TFS_Arrival_Time'], errors='coerce')

# Keep only valid records
df_clean = df[
    (df['TFS_Alarm_Time'].notna()) &
    (df['TFS_Arrival_Time'].notna()) &
    (df['TFS_Arrival_Time'] >= df['TFS_Alarm_Time'])
]

# Reset index (optional)
df_clean = df_clean.reset_index(drop=True)

print("Original rows:", len(df))
print("Cleaned rows:", len(df_clean))


Original rows: 361717
Cleaned rows: 349243


In [11]:
print(df.head())

   _id INCIDENT_NUMBER               Initial_CAD_Event_Type  \
0    1       F23046527                              Medical   
1    2       F23048784                              Medical   
2    3       F23042208  FIHR - Fire -  Highrise Residential   
3    4       F23103667                              Medical   
4    5       F23178577                              Medical   

  Initial_CAD_Event_Call_Type               Final_Incident_Type  \
0                     Medical  34 - Human - Perceived Emergency   
1                     Medical                           Medical   
2              Emergency Fire                         01 - Fire   
3                     Medical                           Medical   
4                     Medical                           Medical   

   Event_Alarm_Level          Call_Source Incident_Station_Area  \
0                  0  03 - From Ambulance                   345   
1                  0  03 - From Ambulance                   443   
2                

In [12]:
df.dtypes

_id                                     int64
INCIDENT_NUMBER                        object
Initial_CAD_Event_Type                 object
Initial_CAD_Event_Call_Type            object
Final_Incident_Type                    object
Event_Alarm_Level                       int64
Call_Source                            object
Incident_Station_Area                  object
Incident_Ward                           int64
Ward_At_Event_Dispatch                float64
Intersection                           object
TFS_Alarm_Time                 datetime64[ns]
TFS_Arrival_Time               datetime64[ns]
Last_TFS_Unit_Clear_Time               object
Persons_Rescued                       float64
geometry                               object
incident_datetime                      object
year                                    int64
month                                   int64
hour                                    int64
day_of_week                             int64
dtype: object

In [13]:
import pandas as pd

# Assuming your dataframe is called 'df'
# Calculate response time in minutes
df['Response_Time_Interval'] = (df['TFS_Arrival_Time'] - df['TFS_Alarm_Time']).dt.total_seconds() / 60

# Or if you want to keep it as a timedelta object
df['Response_Time_Interval_Timedelta'] = df['TFS_Arrival_Time'] - df['TFS_Alarm_Time']

# View the results
print(df[['TFS_Alarm_Time', 'TFS_Arrival_Time', 'Response_Time_Interval']].head())

       TFS_Alarm_Time    TFS_Arrival_Time  Response_Time_Interval
0 2023-04-10 18:36:49 2023-04-10 18:40:18                3.483333
1 2023-04-14 20:46:32 2023-04-14 20:52:01                5.483333
2 2023-04-01 17:31:49 2023-04-01 17:35:40                3.850000
3 2023-07-23 12:10:40 2023-07-23 12:15:15                4.583333
4 2023-12-02 19:14:19 2023-12-02 19:19:22                5.050000


In [14]:
import numpy as np
import pandas as pd

def add_cyclical_features(df, datetime_column='incident_datetime'):
    """
    Add cyclical temporal features from datetime column
    
    Parameters:
    df: DataFrame containing datetime column
    datetime_column: name of datetime column (default: 'incident_datetime')
    
    Returns:
    DataFrame with added cyclical features
    """
    # Ensure datetime column is in datetime format
    df[datetime_column] = pd.to_datetime(df[datetime_column])
    
    # Hour of day (0-23)
    df['hour_sin'] = np.sin(2 * np.pi * df[datetime_column].dt.hour / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df[datetime_column].dt.hour / 24)
    
    # Day of week (0-6, Monday=0, Sunday=6)
    df['day_of_week_sin'] = np.sin(2 * np.pi * df[datetime_column].dt.dayofweek / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df[datetime_column].dt.dayofweek / 7)
    
    # Month (1-12)
    df['month_sin'] = np.sin(2 * np.pi * (df[datetime_column].dt.month - 1) / 12)
    df['month_cos'] = np.cos(2 * np.pi * (df[datetime_column].dt.month - 1) / 12)
    
    return df

# Apply the function
df = add_cyclical_features(df, 'incident_datetime')